In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
trans_with_fraud_lables = dataiku.Dataset("trans_with_fraud_lables")
trans_with_fraud_lables_df = trans_with_fraud_lables.get_dataframe()
activity_with_fraud_labels = dataiku.Dataset("activity_with_fraud_labels")
activity_with_fraud_labels_df = activity_with_fraud_labels.get_dataframe()



In [4]:
trans_with_fraud_lables_df.head(2)

transaction_id                               user_id transaction_date   amount transaction_type merchant_category                 tran_location  is_synthetic
0  0016f03b-a81e-4231-bdab-090587b48264  5c5737fa-271b-42b1-8ef6-72606ceb47bf       2024-10-08   510.81              ATM           Grocery   Ambohidratrimo, Madagascar              0
1  001bd813-6a74-4b86-b77d-30db32fb7eb7  e989afa2-f49c-4928-b688-c91457a001a7       2024-06-10  1761.27           Online       Electronics          Mohammedia, Morocco              0

In [9]:
trans_with_fraud_lables_df[['transaction_type','is_synthetic']].value_counts()

transaction_type  is_synthetic
ATM               0               3026
POS               0               2988
Online            0               2988
Wire              1                501
Online            1                497
dtype: int64

In [10]:
trans_with_fraud_lables_df[['merchant_category','is_synthetic']].value_counts()

merchant_category  is_synthetic
Dining             0               2306
Grocery            0               2280
Electronics        0               2238
Fuel               0               2178
Crypto             1                341
Jewelry            1                338
Luxury             1                319
dtype: int64

In [12]:
trans_with_fraud_lables_df["transaction_date"] = pd.to_datetime(trans_with_fraud_lables_df["transaction_date"])

In [14]:
trans_with_fraud_lables_df.dtypes

transaction_id               object
user_id                      object
transaction_date     datetime64[ns]
amount                      float64
transaction_type             object
merchant_category            object
tran_location                object
is_synthetic                  int64
dtype: object

In [17]:
activity_with_fraud_labels_df.dtypes

log_id                          object
user_id                         object
system_accessed                 object
access_type                     object
act_geo_dev_location            object
access_timestamp        datetime64[ns]
is_synthetic                     int64
dtype: object

In [5]:
activity_with_fraud_labels_df.head(2)

log_id                               user_id   system_accessed access_type      act_geo_dev_location    access_timestamp  is_synthetic
0  010ddb12-3ff5-4df2-a22d-aa8a4ad0b85c  1e929531-913c-4f1f-9e70-c807fec718e5  InvestmentPortal       Login   Al Kharj, Saudi Arabia  2025-04-01 02:58:01             0
1  80e50168-cb1c-442e-9dcb-40b52c4e9f65  0aceb5ab-197c-42d7-9cbb-2fe36cb1dc74         MobileApp        View         Potsdam, Germany  2024-06-03 08:13:55             0

In [18]:
activity_with_fraud_labels_df["a_date"] = activity_with_fraud_labels_df["access_timestamp"].dt.date
trans_with_fraud_lables_df["a_date"] = trans_with_fraud_lables_df["transaction_date"].dt.date


In [19]:
merged_df = trans_with_fraud_lables_df.merge(
    activity_with_fraud_labels_df,
    left_on=["user_id", "a_date"],
    right_on=["user_id", "a_date"],
    how="left"
)

In [23]:
act_tran = merged_df.drop_duplicates()

In [38]:
act_tran.transaction_id.value_counts()

a866862f-da1d-4afe-a5f3-aeca5248f639    3
e68619a9-7cc0-4356-890c-81b569fff94d    3
4aa99126-221a-45ec-888e-65089fda3203    3
99c4673d-8c3e-4772-8c16-1b2cf1280c57    2
605bcc6a-da30-4faa-bbec-12f37d3d8440    2
                                       ..
76fefb08-dc85-47ca-9366-4c119cc5fcf3    1
65b61cd1-2a98-4ddb-b470-2dbaef157991    1
e6a061fc-3cbc-4b8e-b6e7-66e289f88b1c    1
74b3ada5-b672-4997-a367-9b74cc15f047    1
289b68df-fc74-4bbd-83c0-02c9ecf11920    1
Name: transaction_id, Length: 10000, dtype: int64

In [0]:

# Compute recipe outputs
# TODO: Write here your actual code that computes the outputs
# NB: DSS supports several kinds of APIs for reading and writing data. Please see doc.

activity_compliment_tran_df = act_tran # Compute a Pandas dataframe to write into activity_compliment_tran


# Write recipe outputs
activity_compliment_tran = dataiku.Dataset("activity_compliment_tran")
activity_compliment_tran.write_with_schema(activity_compliment_tran_df)